<a href="https://colab.research.google.com/github/AlisonJD/tb_examples/blob/main/Publish_SQL_based_endpoints_on_NGINX_log_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Publish SQL-based endpoints on NGINX log analysis

Based on Tinybird blog post:

https://blog.tinybird.co/2021/01/28/nginx-log-analysis/

In [1]:
#@title Mount your Google Drive to save and use local files
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

% cd "/content/gdrive/My Drive/Colab Notebooks/Tinybird/tb_examples"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/Tinybird/tb_examples


In [2]:
#@title Install Tinybird CLI, utilities and your token
!pip install tinybird-cli -q
!sudo apt-get install jq

import os
import re

if not os.path.isfile('.tinyb'):
  !tb auth

if not os.path.isdir('datasources'):
  !tb init

Reading package lists... Done
Building dependency tree       
Reading state information... Done
jq is already the newest version (1.5+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [3]:
#@title Helper function to write to files
def write_text_to_file(filename, text):
  with open(filename, 'w') as f: f.write(text)

# Worked Example from Blog: 
##Publish SQL-based endpoints on NGINX log analysis

Tinybird can be used to analyze datasets like logs.

Here we use Tinybird to:
 - analyze NGINX logs
 - publish SQL queries as API endpoints

We also show you how to model your Data Sources and API Endpoints to make them faster.


## 1. Build the Data Source from a Sample NGINX Log
Ingest the CSV and transform the columns.

In [4]:
!tb datasource generate https://raw.githubusercontent.com/tinybirdco/log_parsing_template/main/access.log.csv --force

** Generated datasources/access_log.datasource
** => Create it on the server running: $ tb push datasources/access_log.datasource
** => Append data using: $ tb datasource append access.log https://raw.githubusercontent.com/tinybirdco/log_parsing_template/main/access.log.csv`



In [5]:
!tb datasource rm access_log
!tb push datasources/access_log.datasource --force

Do you want to delete access_log? Once deleted, it can't be recovered [y/N]: y
** Data Source 'access_log' deleted
** Processing datasources/access_log.datasource
** Building dependencies
** Running access_log 
** 'access_log' created
** Not pushing fixtures


In [6]:
!tb datasource append access_log 'https://raw.githubusercontent.com/tinybirdco/log_parsing_template/main/access.log.csv'

** 🥚 starting import process
** 🐥 done
** Total rows in access_log: 67351
** Data appended to Data Source 'access_log' successfully!
** Data pushed to access_log


Looking at a single record, we see that we need to add column names and extract information from columns.

In [7]:
!tb sql "select * from access_log limit 1"

---------------------------------------------------------------------
column_00: 10.86.160.14
column_01: -
column_02: -
column_03: [19/Jan/2021:06:25:37
column_04: +0000]
column_05: GET /v0/pipes/pro__ct___v0.json?id_c=18&token=XYZ HTTP/1.1
column_06: 200
column_07: 455
column_08: -
column_09: Apache-HttpClient/4.5.10 (Java/11.0.8)
---------------------------------------------------------------------


Let's do that with a Pipe:

In [8]:
filename="pipes/access_log_transform.pipe"
text='''
DESCRIPTION extract column data from raw access log and name columns
NODE extract_column_data
SQL >
    select
        IPv4StringToNum(column_00) as ip,
        parseDateTimeBestEffort(replaceOne(substring(column_03, 2), ':', ' ')) as time,
        splitByChar(' ', column_05) as tt,
        tt[1] as method,
        tt[2] as path,
        tt[3] as protocol,
        column_06 as status_code,
        column_07 as bytes,
        column_09 as user_agent
    from access_log
'''

write_text_to_file(filename, text)

!tb push pipes/access_log_transform.pipe --force

** Processing pipes/access_log_transform.pipe
** Building dependencies
** Running access_log_transform 
** => Test endpoint at https://api.tinybird.co/v0/pipes/access_log_transform.json
** 'access_log_transform' created
** Not pushing fixtures


In [9]:
!tb sql "select * from access_log_transform limit 1"

-----------------------------------
ip: 769996434
time: 2021-01-19 06:48:04
tt: ['POST', '/HNAP1/', 'HTTP/1.0']
method: POST
path: /HNAP1/
protocol: HTTP/1.0
status_code: 301
bytes: 194
user_agent: -
-----------------------------------


## 2. Publish an Endpoint
Create an Endpoint for the number of requests and average bytes for each IP address. An Endpoint can be consumed by your data products.

In [10]:
filename="pipes/requests_per_endpoint.pipe"
text='''
DESCRIPTION requests per endpoint
NODE grouping
SQL >
    %
    SELECT
        ip,
        count() AS request_count,
        avg(bytes) as avg_bytes
    FROM access_log_transform
    GROUP BY ip
    ORDER BY request_count DESC

NODE endpoint
SQL >
    select IPv4NumToString(ip) as ip_address,
           request_count,
           avg_bytes
    from grouping
'''

write_text_to_file(filename, text)

!tb push pipes/requests_per_endpoint.pipe --force

** Processing pipes/requests_per_endpoint.pipe
** Building dependencies
** Running requests_per_endpoint 
current https://api.tinybird.co/v0/pipes/requests_per_endpoint.json?
    new https://api.tinybird.co/v0/pipes/requests_per_endpoint__checker.json? ... ok
current https://api.tinybird.co/v0/pipes/requests_per_endpoint.json?
    new https://api.tinybird.co/v0/pipes/requests_per_endpoint__checker.json? ... ok
** => Test endpoint at https://api.tinybird.co/v0/pipes/requests_per_endpoint.json
** 'requests_per_endpoint' created
** Not pushing fixtures


In [11]:
TOKEN = !cat ../.tinyb | jq .token 
TOKEN = re.search(r'\"(.*?)\"', TOKEN[0]).group()[1:-1]

In [12]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint.json?token=$TOKEN |head -n 24

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6631  100  6631    0     0  15420      0 --:--:-- --:--:-- --:--:-- 15385
{
	"meta":
	[
		{
			"name": "ip_address",
			"type": "String"
		},
		{
			"name": "request_count",
			"type": "UInt64"
		},
		{
			"name": "avg_bytes",
			"type": "Float64"
		}
	],

	"data":
	[
		{
			"ip_address": "10.86.160.21",
			"request_count": 39624,
			"avg_bytes": 438.6311326468807
		},


In [13]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint.json?token=$TOKEN |tail -n 16

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6631  100  6631    0     0  15565      0 --:--:-- --:--:-- --:--:-- 15565
		{
			"ip_address": "209.141.60.10",
			"request_count": 1,
			"avg_bytes": 264
		}
	],

	"rows": 71,

	"statistics":
	{
		"elapsed": 0.002939912,
		"rows_read": 67351,
		"bytes_read": 1686164
	}
}


In [14]:
!tb sql "SELECT count() FROM requests_per_endpoint"

-----------
| count() |
-----------
|      71 |
-----------


In [15]:
!tb sql "SELECT uniqExact(ip) from access_log_transform"

-----------------
| uniqExact(ip) |
-----------------
|            71 |
-----------------


## 3. Create a Real-Time Endpoint using a Materialized View
For Endpoints serving real-time dashboards, where the user can pick, for example, different date ranges or add filters, it is unacceptable to have to wait. If we had millions of entries a day, the generated endpoints wouldn’t be as fast as needed.

The solution is to use a materialized view.

In [16]:
filename="pipes/requests_per_endpoint_mv.pipe"
text='''
DESCRIPTION materialized view
NODE matview
SQL >
    SELECT
        ip,
        countState() AS request_count,
        avgState(bytes) as avg_bytes
    FROM access_log_transform
    GROUP BY ip

TYPE Materialized
DATASOURCE requests_per_endpoint_ds
'''

write_text_to_file(filename, text)

filename="datasources/requests_per_endpoint_ds.datasource"
text='''
DESCRIPTION materialized view
SCHEMA >
    ip UInt32,
    request_count AggregateFunction(count),
    avg_bytes AggregateFunction(avg, Int32)

ENGINE AggregatingMergeTree
ENGINE_SORTING_KEY ip
'''

write_text_to_file(filename, text)

- The materialized view Pipe uses `countState` and `avgState` for the intermediate states. The Data Source has `AggregateFunction` data types to store those intermediate states.
- The Engine is not the regular `MergeTree` but instead an `AggregatingMergeTree` that tells ClickHouse to aggregate columns on merge operations.
- The sorting key tells ClickHouse which column is used for grouping.

In [17]:
!tb push pipes/requests_per_endpoint_mv.pipe --populate --force

** Processing pipes/requests_per_endpoint_mv.pipe
** Building dependencies
** Running requests_per_endpoint_mv 
** Materialized node 'matview' used the Data Source 'requests_per_endpoint_ds'
** Populating job url https://api.tinybird.co/v0/jobs/8c38a5db-0813-42fd-a240-ecb6c5c973ae
** 'requests_per_endpoint_mv' created
** Not pushing fixtures



`--populate` loads the materialized view with the data already in `access_log_transform`

In [18]:
!tb sql "select uniqExact(ip) from requests_per_endpoint_ds" --stats

** Query took 0.000808764 seconds
** Rows read: 100
** Bytes read: 400 bytes
-----------------
| uniqExact(ip) |
-----------------
|            71 |
-----------------


If we push new data with new IP addresses, the number of rows will rise but if we compare this to the number of rows in the original table, working with this view will be much faster.

Note that this view is updated when you append new data, it does not need to recalculate with all the data - thanks to the intermediate states.

In [26]:
filename="pipes/requests_per_endpoint_fast.pipe"
text='''
DESCRIPTION requests per endpoint fast
NODE grouping
SQL >
    %
    SELECT
        ip,
        countMerge(request_count) AS request_count,
        avgMerge(avg_bytes) as avg_bytes
    FROM requests_per_endpoint_ds
    GROUP BY ip
    ORDER BY request_count DESC

NODE endpoint
SQL >
    select IPv4NumToString(ip) as ip_address,
           request_count,
           avg_bytes
    from grouping
'''

write_text_to_file(filename, text)

!tb push pipes/requests_per_endpoint_fast.pipe --force --no-check

** Processing pipes/requests_per_endpoint_fast.pipe
** Building dependencies
** Running requests_per_endpoint_fast 
** => Test endpoint at https://api.tinybird.co/v0/pipes/requests_per_endpoint_fast.json
** 'requests_per_endpoint_fast' created
** Not pushing fixtures


In [27]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint.json\?token\=$TOKEN | jq .statistics

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6631  100  6631    0     0  14415      0 --:--:-- --:--:-- --:--:-- 14383
{
  "elapsed": 0.003172514,
  "rows_read": 67351,
  "bytes_read": 1686164
}


In [28]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint_fast.json\?token\=$TOKEN | jq .statistics

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6686  100  6686    0     0  15195      0 --:--:-- --:--:-- --:--:-- 15195
{
  "elapsed": 0.000591102,
  "rows_read": 71,
  "bytes_read": 9612
}


The new API Endpoint using the materialized view is faster and reads far less data.

Materialized views could be used for:

- unique ip addresses by day: 
```
select toDate(time) day, uniqState(ip_address) uniq_ip 
from access_log 
group by day
```
- percentile 95 of payload size per hour:
```
select toStartOfHour(time) hour, quantileState(0.95)(ip_address) q95
from access_log_transform 
group by hour
```
- requests per month: 
```
select toStartOfMonth(time) month, countState() requests_count 
from access_log_transform 
group by month
```

